This QC notebook is modeled after the USGS GeoDataPortal [`Zarr` Data Review Tutorial](https://code.usgs.gov/wma/nhgf/geo-data-portal/gdp_data_processing/-/blob/main/workflows/zarr-data-review/tutorial.ipynb?ref_type=heads), but does not contain all elements of their QC process. 

This notebook is designed to check local `zarr` outputs to make sure they meet basic data requirements before submitting for further processing for USGS GeoDataPortal upload.

### Setup

In [1]:
import xarray as xr
from pathlib import Path

In [2]:
# directory with reformatted zarr files
dir = Path('/beegfs/CMIP6/jdpaul3/cmip6_4km_downscaling/adjusted_reformatted')
zarr_stores = list(dir.glob('*.zarr'))
zarr_stores.sort()
zarr_stores

[PosixPath('/beegfs/CMIP6/jdpaul3/cmip6_4km_downscaling/adjusted_reformatted/dtr_EC-Earth3-Veg_historical_adjusted.zarr'),
 PosixPath('/beegfs/CMIP6/jdpaul3/cmip6_4km_downscaling/adjusted_reformatted/dtr_EC-Earth3-Veg_ssp370_adjusted.zarr'),
 PosixPath('/beegfs/CMIP6/jdpaul3/cmip6_4km_downscaling/adjusted_reformatted/dtr_EC-Earth3-Veg_ssp585_adjusted.zarr'),
 PosixPath('/beegfs/CMIP6/jdpaul3/cmip6_4km_downscaling/adjusted_reformatted/pr_EC-Earth3-Veg_historical_adjusted.zarr'),
 PosixPath('/beegfs/CMIP6/jdpaul3/cmip6_4km_downscaling/adjusted_reformatted/pr_EC-Earth3-Veg_ssp126_adjusted.zarr'),
 PosixPath('/beegfs/CMIP6/jdpaul3/cmip6_4km_downscaling/adjusted_reformatted/pr_EC-Earth3-Veg_ssp370_adjusted.zarr'),
 PosixPath('/beegfs/CMIP6/jdpaul3/cmip6_4km_downscaling/adjusted_reformatted/pr_EC-Earth3-Veg_ssp585_adjusted.zarr'),
 PosixPath('/beegfs/CMIP6/jdpaul3/cmip6_4km_downscaling/adjusted_reformatted/tasmax_EC-Earth3-Veg_historical_adjusted.zarr'),
 PosixPath('/beegfs/CMIP6/jdpaul3/cmi

### Review Directory

Each dataset's directory should contain the following files:
- .zattrs
- .zgroup
- .zmetadata
- a directory for each data variable / dimension (ex: 'crs', 'lat', 'time', 'wind_speed')

The following code determines the presence of all these files, and creates a list of successfully opened datasets for further review.

In [3]:
datasets = []

expected_metadata = {'.zattrs', '.zgroup', '.zmetadata'}

for zarr_store in zarr_stores:

    print(f"Checking {zarr_store.name}... ", end='')

    try:
        ds = xr.open_dataset(
            zarr_store,
            engine='zarr',
            backend_kwargs={'consolidated':True},
            chunks={},
            decode_cf=True,
            decode_times=True
        )
        datasets.append(ds)

    except Exception as e:
        print(f"FAILED to open dataset! Error: {e}")
        continue

    missing = []

    # list everything in the zarr directory
    dirs = [str(p.name) for p in zarr_store.iterdir()]

    for metadata in expected_metadata:
        if metadata not in dirs:
            missing.append(metadata)

    for variable in ds.variables:
        if variable not in dirs:
            missing.append(variable)

    if not missing:
        print("PASSED")
    else:
        print("MISSING: ", ", ".join(missing))


print(f"\n\nTotal datasets checked: {len(zarr_stores)}")
print(f"Total datasets successfully opened: {len(datasets)}")

Checking dtr_EC-Earth3-Veg_historical_adjusted.zarr... PASSED
Checking dtr_EC-Earth3-Veg_ssp370_adjusted.zarr... PASSED
Checking dtr_EC-Earth3-Veg_ssp585_adjusted.zarr... PASSED
Checking pr_EC-Earth3-Veg_historical_adjusted.zarr... PASSED
Checking pr_EC-Earth3-Veg_ssp126_adjusted.zarr... PASSED
Checking pr_EC-Earth3-Veg_ssp370_adjusted.zarr... PASSED
Checking pr_EC-Earth3-Veg_ssp585_adjusted.zarr... PASSED
Checking tasmax_EC-Earth3-Veg_historical_adjusted.zarr... PASSED
Checking tasmax_EC-Earth3-Veg_ssp370_adjusted.zarr... PASSED
Checking tasmax_EC-Earth3-Veg_ssp585_adjusted.zarr... PASSED
Checking tasmin_EC-Earth3-Veg_historical_adjusted.zarr... PASSED
Checking tasmin_EC-Earth3-Veg_ssp126_adjusted.zarr... PASSED
Checking tasmin_EC-Earth3-Veg_ssp370_adjusted.zarr... PASSED
Checking tasmin_EC-Earth3-Veg_ssp585_adjusted.zarr... PASSED


Total datasets checked: 14
Total datasets successfully opened: 14


### Time decoding

Check the first 5 values in each dataset's `time` dimension to make sure we are seeing actual date objects and not integers. Seeing integers would suggest that `xarray` was unable to decode the CF-compliant `time` values.

In [4]:
for ds in datasets:
    print(f"\nDataset: {ds.encoding.get('source')}")
    print("First 5 time values:", ds['time'].values[:5])


Dataset: /beegfs/CMIP6/jdpaul3/cmip6_4km_downscaling/adjusted_reformatted/dtr_EC-Earth3-Veg_historical_adjusted.zarr
First 5 time values: [cftime.DatetimeNoLeap(1965, 1, 1, 12, 0, 0, 0, has_year_zero=True)
 cftime.DatetimeNoLeap(1965, 1, 2, 12, 0, 0, 0, has_year_zero=True)
 cftime.DatetimeNoLeap(1965, 1, 3, 12, 0, 0, 0, has_year_zero=True)
 cftime.DatetimeNoLeap(1965, 1, 4, 12, 0, 0, 0, has_year_zero=True)
 cftime.DatetimeNoLeap(1965, 1, 5, 12, 0, 0, 0, has_year_zero=True)]

Dataset: /beegfs/CMIP6/jdpaul3/cmip6_4km_downscaling/adjusted_reformatted/dtr_EC-Earth3-Veg_ssp370_adjusted.zarr
First 5 time values: [cftime.DatetimeNoLeap(2015, 1, 1, 12, 0, 0, 0, has_year_zero=True)
 cftime.DatetimeNoLeap(2015, 1, 2, 12, 0, 0, 0, has_year_zero=True)
 cftime.DatetimeNoLeap(2015, 1, 3, 12, 0, 0, 0, has_year_zero=True)
 cftime.DatetimeNoLeap(2015, 1, 4, 12, 0, 0, 0, has_year_zero=True)
 cftime.DatetimeNoLeap(2015, 1, 5, 12, 0, 0, 0, has_year_zero=True)]

Dataset: /beegfs/CMIP6/jdpaul3/cmip6_4km_do

### File combination

We can test if the files can be successfully combined using `xarray.open_mfdataset()`. 

In [6]:
combined_ds = xr.open_mfdataset(zarr_stores, engine='zarr', combine='by_coords', parallel=True)
combined_ds

<xarray.Dataset> Size: 647GB
Dimensions:      (time: 49640, x: 443, y: 460, ensemble: 4)
Coordinates:
  * time         (time) object 397kB 1965-01-01 12:00:00 ... 2100-12-31 12:00:00
  * x            (x) float64 4kB -8.277e+05 -8.237e+05 ... 9.402e+05 9.442e+05
  * y            (y) float64 4kB 2.476e+06 2.472e+06 ... 6.402e+05 6.362e+05
  * ensemble     (ensemble) object 32B 'EC-Earth3-Veg_r1i1p1f1_historical' .....
    lat          (y, x) float64 2MB dask.array<chunksize=(50, 50), meta=np.ndarray>
    lon          (y, x) float64 2MB dask.array<chunksize=(50, 50), meta=np.ndarray>
    spatial_ref  int64 8B 0
Data variables:
    dtr          (time, y, x, ensemble) float32 162GB dask.array<chunksize=(49640, 50, 50, 2), meta=np.ndarray>
    pr           (time, y, x, ensemble) float32 162GB dask.array<chunksize=(18250, 50, 50, 2), meta=np.ndarray>
    tasmax       (time, y, x, ensemble) float32 162GB dask.array<chunksize=(18250, 50, 50, 3), meta=np.ndarray>
    tasmin       (time, y, x, ensemble) float32 162GB dask.array<chunksize=(18250, 50, 50, 2), meta=np.ndarray>
Attributes:
    contact:        uaf-snap-data-tools@alaska.edu
    creation_date:  2025-09-04T11:12:31
    history:        File was processed by Scenarios Network for Alaska and Ar...